In [53]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
from itertools import product as quite_product
from tqdm.contrib.itertools import product
from collections import Counter
from itertools import cycle

In [54]:
weights_M  = 3
weights_N  = 3
ifmap_M = 224
ifmap_N = 224
channels = 4
ofmap_M = ifmap_M-weights_M+1
ofmap_N = ifmap_N-weights_M+1
ofmap_C = 1

In [55]:
ifmap = np.arange(channels*ifmap_M*ifmap_N).reshape(channels,ifmap_M,ifmap_N)
weights = np.arange(channels*weights_M*weights_N).reshape(channels,weights_M,weights_N)

In [56]:
pads = sliding_window_view(ifmap,window_shape=[1,3,3]).squeeze()
pads.shape

(4, 222, 222, 3, 3)

In [57]:
pad_origin = {}
for p_k, p_i, p_j in product(range(pads.shape[0]), range(pads.shape[1]), range(pads.shape[2])):
    for w_i, w_j in vanilla_product(range(pads.shape[3]), range(pads.shape[4])):
        cur_ifmap = pads[p_k, p_i, p_j, w_i, w_j]
        origin = {"pad_idx": (p_k, p_i, p_j), "weight_idx": (w_i, w_j)}
        if cur_ifmap not in pad_origin.keys():
            pad_origin[cur_ifmap] = [origin]
        else:
            pad_origin[cur_ifmap].append(origin)

100%|██████████| 197136/197136 [00:05<00:00, 38805.10it/s]


In [58]:
weights.shape

(4, 3, 3)

In [60]:
output = []
for k in quite_product(range(channels)):
    pad_completion_counter = Counter()
    for i, j in product(range(ifmap_M), range(ifmap_N)):
        output_added = False
        for assosciated_pads in pad_origin[ifmap[k][i][j]]:
            pad_idx = assosciated_pads['pad_idx']
            pad_completion_counter[pad_idx] += 1
            if pad_completion_counter[pad_idx] == 9:
                output.append(np.einsum('ij,ij', pads[pad_idx], weights[k]))
                output_added = True
        if i>0 and not output_added:
            output.append(0)
                
    output.extend([0]*224)

output.extend([0])
output = np.array(output[1:]).reshape(channels,ifmap_M,ifmap_N)

100%|██████████| 50176/50176 [00:00<00:00, 137170.57it/s]


In [65]:
output[4]


IndexError: index 4 is out of bounds for axis 0 with size 4

1